# Mellea Complete Tutorial

Welcome to the comprehensive interactive tutorial for **Mellea**.
This notebook combines practical usage patterns with core architectural concepts.

**Docs Reference:** [Welcome](https://docs.mellea.ai/overview/mellea-welcome), [Quickstart](https://docs.mellea.ai/overview/quick-start)

In [1]:
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend

# 1. Initialize Mellea with the local Ollama backend and Granite model
# We use a fixed seed for reproducibility in this tutorial.
print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ModelOption.SEED: 42} # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

/Users/anupama/2026/AI/mellea/mellea-IBM/mellea_venv/lib/python3.14/site-packages/granite_common/granite3/granite33/output.py:184: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
  f'{re.escape(CITE_START)}{{"document_id": "(\d+)"}}{re.escape(CITE_END)}'


Initializing Mellea Session...


pulling 6c02683809a8 sha256:6c026: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 2.10G/2.10G [03:20<00:00, 10.6MB/s]
pulling 0f6ec9740c76 sha256:0f6ec:   0%|                                | 0.00/7.08k [00:00<?, ?B/s]
pulling 0f6ec9740c76 sha256:0f6ec: 100%|███████████████████████| 7.08k/7.08k [00:00<00:00, 11.9kB/s]

pulling cfc7749b96f6 sha256:cfc77:   0%|                                | 0.00/11.4k [00:00<?, ?B/s]

pulling cfc7749b96f6 sha256:cfc77: 100%|███████████████████████| 11.4k/11.4k [00:00<00:00, 19.1kB/s]


pulling ba32b08db168 sha256:ba32b:   0%|                                  | 0.00/417 [00:00<?, ?B/s]


                                                                                                                                                                                                                            
                                               

Session started! Model: granite4:micro


## Part 1: Basic Instruction

The core of Mellea is the `instruct` method. You give it a natural language instruction, and it returns a result.

In [2]:
response = m.instruct("Write a short haiku about writing python code.")
print(response)

  0%|                                                                         | 0/2 [00:00<?, ?it/s]

=== 11:49:03-INFO ======
SUCCESS


  0%|                                                                         | 0/2 [00:02<?, ?it/s]

Code in Python flows,
Elegance meets simplicity.
Logic takes its stage.


## Part 2: Components & CBlocks (Core Concepts)

Most LLM libraries just pass strings around. Mellea is different: it uses **Components**.

A `Component` is a structured object Mellea can format for an LLM call.

The minimum you need to define a custom component is:
- `parts()` → list of sub-parts (other Components or CBlocks)
- `format_for_llm()` → string (or TemplateRepresentation)

Then you can run it with `m.act(component)`.

In [6]:
from mellea.core import Component, CBlock
import pydantic

# Define a structured output format using Pydantic
class FlashcardSetOut(pydantic.BaseModel):
    topic: str
    cards: list[dict]

# Define a custom Component
class FlashcardMaker(Component):
    def __init__(self, topic: str, count: int = 5):
        self.topic = topic
        self.count = count

    def parts(self):
        # We don't have nested parts in this simple example.
        return []

    def format_for_llm(self):
        # Returning a STRING is the simplest valid way to represent a Component.
        return (
            "You are a study assistant.\n"
            f"Make {self.count} flashcards about: {self.topic}.\n"
            "Return JSON with keys: topic (string), cards (list of objects).\n"
            "Each card object must have: question, answer, difficulty.\n"
        )

    def _parse(self, computed):
        # In a real component, you might parse the JSON here.
        # For this tutorial, we return the raw value and parse it manually outside.
        return computed.value

# Act on the component
print("Generating Flashcards...")
res = m.act(FlashcardMaker("IBM Quantum hardware", count=3), format=FlashcardSetOut)

print("\nRaw model output:", res.value)

# Parse it back into the Pydantic model
parsed = FlashcardSetOut.model_validate_json(res.value)
print("\nParsed Object:", parsed)

Generating Flashcards...


  0%|                                                   | 0/2 [00:00<?, ?it/s]

=== 11:57:35-INFO ======
SUCCESS


  0%|                                                   | 0/2 [00:21<?, ?it/s]


Raw model output: {
  "topic": "IBM Quantum hardware",
  "cards": [
    {
      "question": "What is the primary function of IBM's quantum hardware?",
      "answer": "The primary function of IBM's quantum hardware is to perform computations using quantum bits (qubits) and leverage principles such as superposition and entanglement for complex problem-solving.",
      "difficulty": "easy"
    },
    {
      "question": "Which company developed the first commercially available quantum computer?",
      "answer": "IBM developed the first commercially available quantum computer, known as IBM Q System One.",
      "difficulty": "medium"
    },
    {
      "question": "What is a qubit in IBM Quantum hardware?",
      "answer": "A qubit (quantum bit) is the fundamental unit of information in quantum computing. Unlike classical bits which can be either 0 or 1, a qubit can exist in multiple states simultaneously due to superposition.",
      "difficulty": "hard"
    }
  ]
}

Parsed Object: top

# The Loop: Instruct-Validate-Repair (Docs: Requirements)

Generative models can be unpredictable. Mellea's power lies in its **Instruct-Validate-Repair** loop. 
You can define **Requirements** (`req`) and **Checks** (`check`) that the output *must* satisfy. If the model fails, Mellea uses a **Strategy** to try again or fix it.

**Docs Reference:** [Requirements](https://docs.mellea.ai/overview/requirements)

### Example: Generating a Helper Email with Constraints
We want to generate an email that:
1.  Has a salutation.
2.  Does **not** use the word 'regards'.
3.  (Optional strict check) Uses only lower-case letters (just to show validation failure and repair).

In [7]:
from mellea.stdlib.sampling import RejectionSamplingStrategy
from mellea.stdlib.requirements import req,check,simple_validate

# using a validation fn to validate your results as well as a check to validate your results
requirements = [
    req("The email should have a salutation"),
    req("Use only lower-case letters", validation_fn=simple_validate(lambda s: s.islower())),
    check("The email should not mention the word 'regards'")
        
]

def write_email(m:mellea.MelleaSession, names :str , notes:str)->str:
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        strategy = RejectionSamplingStrategy(loop_budget = 5),
        user_variables={"name": names, "notes": notes},
        return_sampling_results = True ,
    )
    if email_candidate.success :
        return str(email_candidate.result)
    else :
        print("Expect sub-par output due to failure to meet requirements.")
        return email_candidate.sampling_results[0].value
    
m=mellea.start_session()
print(write_email(m,"Anupama","Anupama has been a great team player, always willing to help others and contribute to group projects."))


=== 11:57:56-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext


  0%|                                                   | 0/5 [00:00<?, ?it/s]

=== 11:57:58-INFO ======
SUCCESS


  0%|                                                   | 0/5 [00:01<?, ?it/s]

subject: acknowledging your excellent teamwork

dear anupama,

i hope this message finds you well. i am writing to highlight how impressed we are with your contributions and dedication as part of our team.

your commitment to being a great teammate is truly remarkable, especially your eagerness to assist others and contribute significantly to group projects. these qualities not only foster a collaborative environment but also greatly enhance the overall productivity and morale within the team.

keep up the fantastic work, anupama!

warm regards,

[your name]


# Advanced: Core Concepts & Generative Programming

**Docs Reference:** [Generative Programming](https://docs.mellea.ai/overview/project-mellea), [Core Concepts](https://docs.mellea.ai/core-concept/generative-slots)

In this section, we touch on more advanced concepts. Mellea treats prompts not just as strings, but as programs with **Slots**, **Context**, and **Agents**.

### Variable Injection (Context)
You saw `user_variables` above. This is part of Mellea's Context Management system, allowing you to inject data safely into prompts.

In [5]:
# Quick example of dynamic variables
response = m.instruct(
    "Translate the following word to Spanish: {{word}}",
    user_variables={"word": "Hello"}
)
print(f"Hello in Spanish: {response}")

  0%|                                                   | 0/2 [00:00<?, ?it/s]

=== 11:56:48-INFO ======
SUCCESS


  0%|                                                   | 0/2 [00:01<?, ?it/s]

Hello in Spanish: The translation of "Hello" in Spanish is "Hola".


In [12]:
# using model options
import mellea
from mellea.backends import ModelOption  #its mellea.backends
from mellea.backends.ollama import OllamaModelBackend
from mellea.backends import model_ids

m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro",model_options={ModelOption.SEED: 42}
    )
)

answer = m.instruct(
    "What is 2x2?",
    model_options={
        "temperature": 0.1,
    },
)

print(str(answer))

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:16:08-INFO ======
SUCCESS


  0%|          | 0/2 [00:02<?, ?it/s]

The mathematical expression "2x2" represents the multiplication of two numbers: 2 and 2. Therefore, 2 multiplied by 2 equals 4. In other words, if you have two groups with two items in each group, the total number of items would be four.


## Generative Slots

``GenerativeSlot`` is a function whose implementation is provieded by an LLM.In Mellea you define these using ``@generative`` decorator

In [13]:
# from mellea.std.base import ChatContext is old 
from mellea.stdlib.context import ChatContext
from mellea import start_session
from mellea.backends import ModelOption
from mellea import generative   

# ChatContext is used to maintain conversation history across multiple model calls. 
# Unlike SimpleContext (the default), which resets the chat history on each call, 
# ChatContext behaves like a chat history where previous messages are remembered.

@generative
def grade_syntax(code: str) -> int:
    """ Grade the code based on correct implementation of the function
    args:
        code: str (to be graded)
    returns:
        int : a grade between 1(worst) and 10(best)
    """
codes = (
    " def add(a, b):\n    return a + b",
    "def subtract(a,b): cout<<a-b<<endl",
    "int multiply(int a,int b) { return a*b}"
)
m = start_session(ctx = ChatContext() , model_options = {ModelOption.MAX_NEW_TOKENS: 10})
for code in codes :
    grade = grade_syntax(m, code = code)
    print("grade = ",grade)

=== 16:16:08-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=ChatContext, model_options={'@@@max_new_tokens@@@': 10}
grade =  10
grade =  2
grade =  8


### Advanced: Using Generative slots to Provide Compositionality Across Module Boundaries

In [14]:
# file: https://github.com/generative-computing/mellea/blob/main/docs/examples/tutorial/compositionality_with_generative_slots.py#L1-L18
from mellea import generative
from typing import Literal

# The Summarizer Library
@generative
def summarize_meeting(transcript: str) -> str:
  """Summarize the meeting transcript into a concise paragraph of main points."""

@generative
def summarize_contract(contract_text: str) -> str:
  """Produce a natural language summary of contract obligations and risks."""

@generative
def summarize_short_story(story: str) -> str:
  """Summarize a short story, with one paragraph on plot and one paragraph on broad themes."""


# The Decision Aides Library
@generative
def propose_business_decision(summary: str) -> str:
  """Given a structured summary with clear recommendations, propose a business decision."""

@generative
def generate_risk_mitigation(summary: str) -> str:
  """If the summary contains risk elements, propose mitigation strategies."""

@generative
def generate_novel_recommendations(summary: str) -> str:
  """Provide a list of novel recommendations that are similar in plot or theme to the short story summary."""

# Compose the libraries.
@generative
def has_structured_conclusion(summary: str) -> Literal["yes", "no"]:
  """Determine whether the summary contains a clearly marked conclusion or recommendation."""

@generative
def contains_actionable_risks(summary: str) -> Literal["yes", "no"]:
  """Check whether the summary contains references to business risks or exposure."""

@generative
def has_theme_and_plot(summary: str) -> Literal["yes", "no"]:
  """Check whether the summary contains both a plot and thematic elements."""


In [15]:
m = start_session()
transcript = """Meeting Transcript: Market Risk Review -- Self-Sealing Stembolts Division
Date: December 1, 3125
Attendees:

Karen Rojas, VP of Product Strategy

Derek Madsen, Director of Global Procurement

Felicia Zheng, Head of Market Research

Tom Vega, CFO

Luis Tran, Engineering Liaison

Karen Rojas:
Thanks, everyone, for making time on short notice. As you've all seen, we've got three converging market risks we need to address: tariffs on micro-carburetors, increased adoption of the self-interlocking leafscrew, and, believe it or not, the "hipsterfication" of the construction industry. I need all on deck and let's not waste time. Derek, start.

Derek Madsen:
Right. As of Monday, the 25% tariff on micro-carburetors sourced from the Pan-Alpha Centauri confederacy is active. We tried to pre-purchase a three-month buffer, but after that, our unit cost rises by $1.72. That's a 9% increase in the BOM cost of our core model 440 stembolt. Unless we find alternative suppliers or pass on the cost, we're eating into our already narrow margin.

Tom Vega:
We cannot absorb that without consequences. If we pass the cost downstream, we risk losing key mid-tier OEM clients. And with the market already sniffing around leafscrew alternatives, this makes us more vulnerable.

Karen:
Lets pause there. Felicia, give us the quick-and-dirty on the leafscrew.

Felicia Zheng:
It's ugly. Sales of the self-interlocking leafscrew—particularly in modular and prefab construction—are up 38% year-over-year. It's not quite a full substitute for our self-sealing stembolts, but they are close enough in function that some contractors are making the switch. Their appeal? No micro-carburetors, lower unit complexity, and easier training for install crews. We estimate we've lost about 12% of our industrial segment to the switch in the last two quarters.

Karen:
Engineering, Luis; your take on how real that risk is?

Luis Tran:
Technically, leafscrews are not as robust under high-vibration loads. But here's the thing: most of the modular prefab sites don not need that level of tolerance. If the design spec calls for durability over 10 years, we win. But for projects looking to move fast and hit 5-year lifespans? The leafscrew wins on simplicity and cost.

Tom:
So they're eating into our low-end. That's our volume base.

Karen:
Exactly. Now let's talk about this last one: the “hipsterfication” of construction. Felicia?

Felicia:
So this is wild. We're seeing a cultural shift in boutique and residential construction—especially in markets like Beckley, West Sullivan, parts of Osborne County, where clients are requesting "authentic" manual fasteners. They want hand-sealed bolts, visible threads, even mismatched patinas. It's an aesthetic thing. Function is almost secondary. Our old manual-seal line from the 3180s? People are hunting them down on auction sites.

Tom:
Well, I'm glad I don't have to live in the big cities... nothing like this would ever happen in downt-to-earth places Brooklyn, Portland, or Austin.

Luis:
We literally got a request from a design-build firm in Keough asking if we had any bolts “pre-distressed.”

Karen:
Can we spin this?

Tom:
If we keep our vintage tooling and market it right, maybe. But that's niche. It won't offset losses in industrial and prefab.

Karen:
Not yet. But we may need to reframe it as a prestige line—low volume, high margin. Okay, action items. Derek, map alternative micro-carburetor sources. Felicia, get me a forecast on leafscrew erosion by sector. Luis, feasibility of reviving manual seal production. Tom, let's scenario-plan cost pass-through vs. feature-based differentiation.

Let's reconvene next week with hard numbers. Thanks, all."""
summary = summarize_meeting(m, transcript=transcript)

if contains_actionable_risks(m, summary=summary) == "yes":
    mitigation = generate_risk_mitigation(m, summary=summary)
    print(f"Mitigation: {mitigation}")
else:
    print("Summary does not contain actionable risks.")
if has_structured_conclusion(m, summary=summary) == "yes":
    decision = propose_business_decision(m, summary=summary)
    print(f"Decision: {decision}")
else:
    print("Summary lacks a structured conclusion.")

=== 16:16:11-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext
Mitigation: Based on the summary provided, here are potential mitigation strategies for each market risk discussed in the Self-Sealing Stembolts Division: 

1. **25% Tariff on Micro-Carburetors**: Explore alternative suppliers that may not be subject to the same tariff rates or consider sourcing from regions with lower tariffs. Additionally, evaluate if there's a cost-effective way to pass these increased costs onto end consumers or explore bulk purchasing agreements with suppliers.

2. **Increased Adoption of Self-Interlocking Leaf Screws**: Differentiate your product by offering higher quality and superior features that can outshine competitors in this category. Consider investing in research and development for innovative products, marketing efforts to highlight the benefits over self-interlocking leaf screws, or even forming strategic partnerships with tool manufacturers wh

## **Mify Objects**

```MObject``` storing data along side with its relevant operations(tools).This allows LLMs to interact with both tthe data and methods in a unifiied structured manner , simplifies exposinhg only specific fields and methods for LLMs to interact with.

```Mobject``` pattern also provides a way of evolvoing exsisting classical codebases into generative programs.

In [16]:
import mellea
# old was mellea.stdlib.mify 
from mellea.stdlib.components.mify import mify, MifiedProtocol
import pandas as pd
from io import StringIO

@mify(fields_include = {"table"} , template = "{{table}}")
class myDB:
    table : str = """| Store      | Sales   |
                    | ---------- | ------- |
                    | Northeast  | $250    |
                    | Southeast  | $80     |
                    | Midwest    | $420    |"""

    def transpose(self):
        pd.read_csv(
            StringIO(self.table) , 
            sep = "|" ,
            skipinitialspace= True ,
            header = 0 ,
            index_col = False 
        )

m = mellea.start_session()
db = myDB()
assert isinstance(db , MifiedProtocol)
answer = m.query( db , " What is the total sales of the east ?")
print(str(answer))

=== 16:16:41-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext
The total sales for the East region, which includes both the Northeast and Southeast stores, can be calculated by adding their individual sales.

Northeast store's sales = $250
Southeast store's sales = $80

Total sales in the East = Sales in Northeast + Sales in Southeast = $250 + $80 = $330


### How to find new updates in the code base that cause you to use older imports and throws errors ?
```
(mellea) ashoksharma@Ashoks-MacBook-Pro Mellea_IBM % cd mellea
(mellea) ashoksharma@Ashoks-MacBook-Pro mellea % find . -name "mify.py"
./mellea/stdlib/components/mify.py
./docs/examples/mify/mify.py 
```

In [ ]:
# Working with documents

from mellea.stdlib.components.docs.richdocument import RichDocument
# Note: This requires the docling package installed

rd = RichDocument.from_document_file("https://arxiv.org/pdf/1906.04043")

In [23]:
# Example of working with a table from a document
from mellea.stdlib.components.docs.richdocument import Table
table1 : Table = rd.get_tables()[0] 
print(table1.to_markdown())

Table 1: Cross-validated results of fake-text discriminators. Distributional information yield a higher informativeness than word-features in a logistic regression.

| Feature                              | AUC         |
|--------------------------------------|-------------|
| Bag of Words                         | 0.63 ± 0.11 |
| (Test 1 - GPT-2) Average Probability | 0.71 ± 0.25 |
| (Test 2 - GPT-2) Top-K Buckets       | 0.87 ± 0.07 |
| (Test 1 - BERT) Average Probability  | 0.70 ± 0.27 |
| (Test 2 - BERT) Top-K Buckets        | 0.85 ± 0.09 |


In [24]:
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend
from mellea import start_session

print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ ModelOption.SEED : 42, ModelOption.MAX_NEW_TOKENS : 500} # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

op = ("Add a column 'Model' that extracts which model was used or 'None' if none")

for seed in [x*12 for x in range(5)]:
    table2 = m.transform(table1,op, model_options={ ModelOption.SEED : seed})
    if isinstance(table2 , Table):
        print(table2.to_markdown())
        break
    else :
        print(f"====== TRYING AGAIN AS OUTPUT WAS NOT USEFUL")

Initializing Mellea Session...
Session started! Model: granite4:micro
=== 16:18:32-INFO ======
Tools for call: dict_keys(['from_markdown', 'to_markdown', 'transpose'])
====== TRYING AGAIN AS OUTPUT WAS NOT USEFUL
=== 16:18:55-INFO ======
Tools for call: dict_keys(['from_markdown', 'to_markdown', 'transpose'])
=== 16:19:10-INFO ======
added a tool message from transform to the context
====== TRYING AGAIN AS OUTPUT WAS NOT USEFUL
=== 16:19:10-INFO ======
Tools for call: dict_keys(['from_markdown', 'to_markdown', 'transpose'])
=== 16:19:21-INFO ======
added a tool message from transform to the context
| Feature                              | AUC         | Model   |
|--------------------------------------|-------------|---------|
| Bag of Words                         | 0.63 ± 0.11 | None    |
| (Test 1 - GPT-2) Average Probability | 0.71 ± 0.25 | GPT-2   |
| (Test 2 - GPT-2) Top-K Buckets       | 0.87 ± 0.07 | GPT-2   |
| (Test 1 - BERT) Average Probability  | 0.70 ± 0.27 | BERT    |
| (T

In [25]:
##MObjects methods as tools
# when an object is mified all methods with a docstring get registered as tools for LLM calls
# here only the from makrdown method will be exposed as tool to the llm (using funcs_include and funcs_exlude)

from mellea.stdlib.components.mify import mify

@mify(funcs_include = {"from_markdown"})
class myDocLoader:
    def __init__(self) -> None :
        self.content = " "
    
    @classmethod
    def from_markdown(cls , text : str) -> "MyDocLoader" :
        doc = MyDocLoader()
        #Your parsing func goes below
        doc.content = text
        return doc
    def dummy(self) -> str:
        return " I am excluded "



In [26]:
table1_v1 = m.transform(table1 , "Transpose the table.")
table1_v2 = table1.transpose()
print(table1_v1.to_markdown())
print(table1_v2.to_markdown())

=== 16:19:22-INFO ======
Tools for call: dict_keys(['from_markdown', 'to_markdown', 'transpose'])


Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


=== 16:19:30-WARNING ======
the transform of <mellea.stdlib.components.docs.richdocument.Table object at 0x3734e9370> with transformation description 'Transpose the table.' resulted in a tool call with no generated arguments; consider calling the function `transpose` directly
=== 16:19:30-INFO ======
added a tool message from transform to the context


Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


|         | 0            | 1                                    | 2                              | 3                                   | 4                             |
|---------|--------------|--------------------------------------|--------------------------------|-------------------------------------|-------------------------------|
| Feature | Bag of Words | (Test 1 - GPT-2) Average Probability | (Test 2 - GPT-2) Top-K Buckets | (Test 1 - BERT) Average Probability | (Test 2 - BERT) Top-K Buckets |
| AUC     | 0.63 ± 0.11  | 0.71 ± 0.25                          | 0.87 ± 0.07                    | 0.70 ± 0.27                         | 0.85 ± 0.09                   |
|         | 0            | 1                                    | 2                              | 3                                   | 4                             |
|---------|--------------|--------------------------------------|--------------------------------|-------------------------------------|-------------------

### Debugging: Session Reset & Last Turn

These are the 3 things you should check *all the time* when learning:

1. `m.last_prompt()` → what did you actually send?
2. `m.ctx.last_turn()` → last user+assistant pair
3. `m.ctx.view_for_generation()` (advanced) → what context will be sent next

In [20]:
from mellea.backends import ModelOption
from mellea import start_session

print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ ModelOption.SEED: 42, ModelOption.MAX_NEW_TOKENS:500 } # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

Initializing Mellea Session...
Session started! Model: granite4:micro


In [27]:
# Debugging commands
print("--- last_prompt ---")
print(m.last_prompt())

print("\n--- last_turn ---")
print(m.ctx.last_turn() if hasattr(m, "ctx") else "no ctx")

--- last_prompt ---
[{'role': 'user', 'content': 'For the following table please execute the following transformation. The table shown here is a representation of the actual `Table` object any tools will be run against.\n\nThe output you provide should be a markdown table or a function call that results in a `Table` object. \n\nIf using a tool, use the tool whose name most closely matches the transformation request:\nTable:\nTable 1: Cross-validated results of fake-text discriminators. Distributional information yield a higher informativeness than word-features in a logistic regression.\n\n| Feature                              | AUC         |\n|--------------------------------------|-------------|\n| Bag of Words                         | 0.63 ± 0.11 |\n| (Test 1 - GPT-2) Average Probability | 0.71 ± 0.25 |\n| (Test 2 - GPT-2) Top-K Buckets       | 0.87 ± 0.07 |\n| (Test 1 - BERT) Average Probability  | 0.70 ± 0.27 |\n| (Test 2 - BERT) Top-K Buckets        | 0.85 ± 0.09 |\n\nTransform

## Context Management

Mellea manages context using two complementary mechanisms:
- Components themselves, which generally contain all of the context needed for a single-turn request. MObjects manage context using fields and methods, and Instructions have a grounding_context for RAG-style requests.
- The Context, which stores and represents a (sometimes partial) history of all previous requests to the LLM made during the current session.

In [29]:
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend
from mellea.stdlib.context import SimpleContext, ChatContext 
#other here is chat Context
from mellea import start_session


# whenever we use start session we actually instiantiate Mellea with a default inference engine ,default model choice and a default Context Manager which is a simple Context Manager 
print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
    ),
    ctx = SimpleContext()
)
print("Session started! Model: granite4:micro")

Initializing Mellea Session...
Session started! Model: granite4:micro


The ```SimpleContext``` — which is the only context we have used so far — is a context manager that resets the chat message history on each model call. That is, the model’s context is entirely determined by the current Component. Mellea also provides a ```ChatContext```, which behaves like a chat history. We can use the ChatContext to interact with chat models:

In [30]:
from mellea import start_session

m = mellea.start_session(ctx=ChatContext())
m.chat("Make up a math problem.")
m.chat("Solve your math problem.")

=== 16:20:58-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=ChatContext


mellea.Message(role="assistant", content="Sure, let's solve it step by step.

From equations (1) and (2), we can set them equal to each other:

1/2 * Y = 1/3 * Z

Multiplying both sides of the equation by 6 gives us:

3Y = 2Z 

Now express Z in terms of Y:

Z = (3/2)Y

Next, substitute this expression for Z back into our sum equation:

X + Y + (3/2)Y = 45

Combine like terms:

(1 + 1 + 3/2)Y = 45
(5/2)Y = 45

Now solve for Y by multiplying both sides of the equation by 2/5:

Y = 90/5
Y = 18

Substitute Y back into our expression for Z:

Z = (3/2)*18
Z = 27

Finally, substitute Y back into either equation (1) or (2) to find X. Let's use equation (1):

X = 1/2 * Y
X = 1/2 * 18
X = 9

So the three parts are:
- First part (X): 9
- Second part (Y): 18 
- Third part (Z): 27", images="[]", documents="[]")

## Building Agents

Mellea supports building agents like ReACT/Reasoning Loop agents. 
See `demo_agent.py` for a full implementation of a ReACT agent.

## Interoperability

- **MCP (Model Context Protocol)**: Expose Mellea tools to other AI clients. (See `demo_mcp.py`)
- **crewAI Integration**: Mellea can be used as a backend for crewAI agents.

# Prompt Engineering
Most backends operate on text. For these backends/models, Mellea has an opinionated stance on how to transform Python objects into text: the TemplateFormatter.

In [ ]:
import docs.examples.generative_slots.generative_slots_with_requirements
import asyncio.base_events
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend

# 1. Initialize Mellea with the local Ollama backend and Granite model
# We use a fixed seed for reproducibility in this tutorial.
print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ModelOption.SEED: 42} # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

Initializing Mellea Session...
Session started! Model: granite4:micro


In [4]:
from mellea.core import Component, CBlock
import pydantic

# Define a structured output format using Pydantic
class FlashcardSetOut(pydantic.BaseModel):
    topic: str
    cards: list[dict]

# Define a custom Component
class FlashcardMaker(Component):
    def __init__(self, topic: str, count: int = 5):
        self.topic = topic
        self.count = count

    def parts(self):
        # We don't have nested parts in this simple example.
        return []
    
    # Along with a template, each class/object needs to define the arguments that will be supplied when rendering the template.This happens here format_for_llm
    # It returns either a string or a TemplateRepresentation(obj, args, template).

    def format_for_llm(self):
        # Returning a STRING is the simplest valid way to represent a Component.
        return (
            "You are a study assistant.\n"
            f"Make {self.count} flashcards about: {self.topic}.\n"
            "Return JSON with keys: topic (string), cards (list of objects).\n"
            "Each card object must have: question, answer, difficulty.\n"
        )

    def _parse(self, computed):
        # In a real component, you might parse the JSON here.
        # For this tutorial, we return the raw value and parse it manually outside.
        return computed.value

# Act on the component
# m.act(component) → model sees the component as the “action”

# m.instruct("...", grounding_context={...}) → model sees your instruction plus the rendered grounding context (using your templates).
print("Generating Flashcards...")
res = m.act(FlashcardMaker("OOP in Python", count=3), format=FlashcardSetOut)

print("\nRaw model output:", res.value)

# Parse it back into the Pydantic model
parsed = FlashcardSetOut.model_validate_json(res.value)
print("\nParsed Object:", parsed)

Generating Flashcards...


  0%|          | 0/2 [00:00<?, ?it/s]

=== 06:25:44-INFO ======
SUCCESS


  0%|          | 0/2 [00:15<?, ?it/s]


Raw model output: {
  "topic": "OOP in Python",
  "cards": [
    {
      "question": "What is the role of 'class' keyword in OOP?",
      "answer": "'class' keyword is used to define a new class and introduce its members like attributes, methods.",
      "difficulty": "easy"
    },
    {
      "question": "How are objects created from classes defined by a user?",
      "answer": "Objects are created using the syntax: ClassName(arguments)",
      "difficulty": "medium"
    },
    {
      "question": "What is encapsulation in OOP?",
      "answer": "Encapsulation is the process of hiding the internal state of an object and exposing only what's necessary, often done by creating private properties.",
      "difficulty": "hard"
    }
  ]
}

Parsed Object: topic='OOP in Python' cards=[{'question': "What is the role of 'class' keyword in OOP?", 'answer': "'class' keyword is used to define a new class and introduce its members like attributes, methods.", 'difficulty': 'easy'}, {'question': 'H

In [ ]:
import mellea
from mellea.core import TemplateRepresentation, ModelOutputThunk
from mellea.stdlib.components.docs.richdocument import RichDocument
import pydantic

class AbstractOut(pydantic.BaseModel):
    title_guess : str 
    one_paragraph_abtract : str

#
class PoorDocumentExtractor(RichDocument):
    
    """ takes only heading and gives an appropriate summary based on that """
   
   # just formatting titles nicely
    def format_for_llm(self) -> TemplateRepresentation :
        titles = []
        md = self.to_markdown()
        for line in md.splitlines():
            if line.startswith("#"):
                titles.append(line.lstrip("#").strip())

        template = (
            "You will see section titles from a document.\n"
            "Infer what the document is about.\n\n"
            "{% for t in titles %}- {{ t }}\n{% endfor %}"
        )

        return TemplateRepresentation( obj = self , args = {"titles" : titles} , template = template)
        


In [7]:
m = mellea.start_session()
source = "https://arxiv.org/pdf/1906.04043"

doc = PoorDocumentExtractor.from_document_file(source)

# Composed prompt: small instruction + grounded doc view
out = m.instruct(
    "Based only on [DOC_TITLES], guess the paper topic and write a one-paragraph abstract.\n"
    "Return JSON with: title_guess, one_paragraph_abstract.",
    grounding_context={"DOC_TITLES": doc},
    format=AbstractOut,
)

# out might be ModelOutputThunk; print raw + parsed
if isinstance(out, ModelOutputThunk):
    print("RAW:\n", out.value)
    parsed = AbstractOut.model_validate_json(out.value)
    print("\nPARSED:\n", parsed)

=== 06:37:05-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext


  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: unhashable type: 'list'